In [1]:
%reset -f

In [2]:
# !pip install brian2

In [3]:
!pip install torch

In [4]:
!pip install pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.1/778.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 27.6 MB/s eta 0:00:00


In [5]:
!pip install snntorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 6.0 MB/s eta 0:00:00


In [6]:
!pip install torchsummary

## set seeds for PyTorch and Numpy to ensure reproducibility:

In [7]:
import torch
import random
import numpy as np

# Set seeds for Python, Numpy, and Torch for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Additional steps if you're using GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [8]:
# imports
import pickle
import matplotlib.animation as animation
from scipy.integrate import simps
import torch
torch.cuda.empty_cache()
import os, sys, time, datetime, json, random
import snntorch as snn
from snntorch import spikeplot as splt
from snntorch import spikegen
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import os
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision import utils as utls
from snntorch import utils
from snntorch import surrogate
import numpy as np
import math
from sklearn.metrics import auc
from torchsummary import summary
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import torch.nn as nn
from scipy.stats import entropy
from scipy.special import kl_div
from torch.autograd import Variable
# import spikeflow as snn

In [9]:
# torch.autograd.set_detect_anomaly(True)

In [10]:
# /////////////////////# Building the Autoencoder
#-------------------DataLoaders.  using the MNIST dataset

# dataloader arguments
batch_size = 250
data_path='/data/mnist'

dtype = torch.float
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


# /////////////////////////////////# Define a transform
input_size = 32 # resizing the original MNIST from 28 to 32

transform = transforms.Compose([
            transforms.Resize((input_size, input_size)),
            transforms.Grayscale(),
            transforms.ToTensor(),
            transforms.Normalize((0,), (1,))])

#------------------------------------------- Load MNIST
# Training data
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Testing data
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)



100%|██████████| 9912422/9912422 [00:00<00:00, 111665707.58it/s]


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 41756530.10it/s]


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 31887660.15it/s]


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 17688513.25it/s]


Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



In [11]:
# creating directories where we can save the original and reconstructed images for training and testing:
# create training/ and testing/ folders in the chosen path
if not os.path.isdir('figures/training'):
    os.makedirs('figures/training')
if not os.path.isdir('figures/binarytraining'):
    os.makedirs('figures/binarytraining')

if not os.path.isdir('figures/testing'):
    os.makedirs('figures/testing')
if not os.path.isdir('figures/binarytesting'):
    os.makedirs('figures/binarytesting')


if not os.path.isdir('Saved_Trained_Checkpoints/'):
    os.makedirs('Saved_Trained_Checkpoints/')

if not os.path.isdir('Output_Spikes/'):
    os.makedirs('Output_Spikes/')

if not os.path.isdir('Enc_syn_Spikes/'):
    os.makedirs('Enc_syn_Spikes/')


if not os.path.isdir('Intermediate_Lyrs/'):
    os.makedirs('Intermediate_Lyrs/')



In [12]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using {torch.cuda.get_device_name()} ({device})")
else:
    device = torch.device("cpu")
    print("Using CPU")


Using CPU


### To manipulate and test (Working for Encoder and Decoder outputs):

In [ ]:
# class SAE(nn.Module):
#     def __init__(self):
#         super().__init__()


#         # Encoder
#         self.encoder = nn.Sequential(
#                             nn.Conv2d(1, 32, 3,padding = 1,stride=2), # Conv Layer 1
#                             # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1,
#                                 # bias=True, padding_mode='zeros',  device=None, dtype=None)
#                             nn.BatchNorm2d(32),
#                             snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True,threshold=thresh),
#                             # snn.Synaptic(alpha=alpha, beta=beta_syn, spike_grad=spike_grad, init_hidden=True,threshold=thresh), #SNN TORCH LIF NEURON
#                             nn.Conv2d(32, 64, 3,padding = 1,stride=2), # Conv Layer 2
#                             nn.BatchNorm2d(64),
#                             # snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True,threshold=thresh),
#                             snn.Synaptic(alpha=alpha, beta=beta_syn, spike_grad=spike_grad, init_hidden=True,threshold=thresh), #SNN TORCH LIF NEURON
#                             # snn.Alpha(alpha=alpha1, beta=beta1, spike_grad=spike_grad, init_hidden=True,threshold=thresh),
#                             nn.Conv2d(64, 128, 3,padding = 1,stride=2), # Conv Layer 3
#                             nn.BatchNorm2d(128),
#                             # snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True,threshold=thresh),
#                             snn.Synaptic(alpha=alpha, beta=beta_syn, spike_grad=spike_grad, init_hidden=True,threshold=thresh), #SNN TORCH LIF NEURON
#                             # snn.Alpha(alpha=alpha11, beta=beta11, spike_grad=spike_grad, init_hidden=True,threshold=thresh),
#                             nn.Flatten(start_dim = 1, end_dim = 3), #Flatten convolutional output
#                             nn.Linear(128*4*4, latent_dim), # Fully connected linear layer
#                             snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, output=True,threshold=thresh)
#                             # snn.Synaptic(alpha=alpha, beta=beta_syn, spike_grad=spike_grad, init_hidden=True, output=True,threshold=thresh)
#                             )


#         self.latent_dim = latent_dim #dimensions of the encoded z-space data

#         #ve from the flattened encoded representation (latent_dim) back to a tensor representation to
#             # use in transposed convolution.
#           # To do so, we need to run an additional fully-connected linear layer transforming the data back into a tensor of 128 x 4 x 4:

#         self.linearNet= nn.Sequential(
#                                       nn.Linear(latent_dim,128*4*4),
#                                       snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, output=True,threshold=thresh))
#                                       #snn.Synaptic(alpha=alpha, beta=beta_syn, spike_grad=spike_grad, init_hidden=True, output=True,threshold=thresh))




#       #  The decoder, with three transposed convolutional (nn.ConvTranspose2d) layers and one linear output layer.
#       # Although we converted the latent data back into tensor form for convolution, we still need to Unflatten it to a tensor of 128 x 4 x 4,
#         # as the input to the network is 1 dimensional.  This is done using nn.Unflatten in the first line of the Decoder:
#         # Decoder:
#         self.decoder = nn.Sequential(
#                             nn.Unflatten(1,(128,4,4)), #Unflatten data from 1 dim to tensor of 128 x 4 x 4
#                             snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True,threshold=thresh),
#                             # snn.Synaptic(alpha=alpha, beta=beta_syn, spike_grad=spike_grad, init_hidden=True,threshold=thresh),
#                             nn.ConvTranspose2d(128, 64, 3,padding = 1,stride=(2,2),output_padding=1),
#                             nn.BatchNorm2d(64),
#                             # snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True,threshold=thresh),
#                             snn.Synaptic(alpha=alpha, beta=beta_syn, spike_grad=spike_grad, init_hidden=True,threshold=thresh),
#                             # snn.Alpha(alpha=alpha2, beta=beta2, spike_grad=spike_grad, init_hidden=True,threshold=thresh),
#                             nn.ConvTranspose2d(64, 32, 3,padding = 1,stride=(2,2),output_padding=1),
#                             nn.BatchNorm2d(32),
#                             # snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True,threshold=thresh),
#                             snn.Synaptic(alpha=alpha, beta=beta_syn, spike_grad=spike_grad, init_hidden=True,threshold=thresh),
#                             # snn.Alpha(alpha=alpha22, beta=beta22, spike_grad=spike_grad, init_hidden=True,threshold=thresh),
#                             nn.ConvTranspose2d(32, 1, 3,padding = 1,stride=(2,2),output_padding=1),
#                             snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True,output=True,threshold=20000) #so membrane can be trained
#                             # snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True,output=True,threshold=thresh) #---------------------------------------------- ADDED
#                             # snn.Synaptic(alpha=alpha, beta=beta_syn, spike_grad=spike_grad, init_hidden=True,output=True,threshold=20000) #so membrane can be trained
#                             )
#         # final Leaky layer, our spiking threshold (thresh) is set extremely high. This is a neat trick in snnTorch, which allows the neuron
#         # membrane in the final layer to continuously be updated, without ever reaching a spiking threshold.

#         # using the membrane potential output from the final layer for the image reconstruction.
#             # snnTorch allows us to use either the spikes or membrane potential of each neuron in training.


#     def forward(self, x):
#         utils.reset(self.encoder) #need to reset the hidden states of LIF
#         utils.reset(self.decoder)
#         utils.reset(self.linearNet)

#     #-----------------------------encode
#         spk_mem=[];
#         spk_rec=[];
#         spk_rec_syn=[];
#         encoder_mem=[];
#         spk_rec_dec=[];
#         spk_mem_dec=[];
#         for step in range(num_steps): #for t in time
#             spk_x, mem_x = self.encode(x) #Output spike trains and neuron membrane states
#             # print("Size of = self.encode(x):", self.encode(x).size())
#             # print("len of = self.encode(x):", len(self.encode(x)))
#             spk_rec.append(spk_x)
#             spk_mem.append(mem_x)

#         spk_rec=torch.stack(spk_rec,dim=2) # stack spikes in second tensor dimension
#         spk_mem=torch.stack(spk_mem,dim=2) # stack membranes in second tensor dimension
# # torch.stack joins (concatenates) a sequence of tensors (two or more tensors) along a new dimension.
#         # print("Size of spk_rec:", spk_rec.size())   # Size of spk_rec: torch.Size([250, 32, 5])
#         # print("Size of spk_mem:", spk_mem.size()) # Size of spk_mem: torch.Size([250, 32, 5])
#         # out_en = spk_rec[:,:,-1]     #//////////////////////////////////////ADDED---------------- shows  batch size (250) different examples on (32) channel number
#         out_en = spk_rec[0, :,:] #//////////////////////////////////////ADDED------------ latent dim (32) and time (num_steps)(5)
#         # print("Size of out_en:", out_en.size()) #Size of out_en: torch.Size([250, 32])

#     #------------------------------decode
#         spk_mem2=[];
#         spk_rec2=[];
#         decoded_x=[];
#         spk_x_dec=[];
#         mem_x_dec=[];
#         for step in range(num_steps): #for t in time
#             # spk_x_dec, mem_x_dec = self.decode(spk_rec[...,step]) #//////////////////////////ADDED
#             x_recon, x_mem_recon = self.decode(spk_rec[...,step])

#             spk_rec2.append(x_recon)
#             spk_mem2.append(x_mem_recon)

#             # spk_rec_dec.append(spk_x_dec)   #//////////////////////////ADDED Size of spk_rec_dec: torch.Size([250, 1, 5, 32, 32])
#             # spk_mem_dec.append(mem_x_dec)   #//////////////////////////ADDED

#         # spk_rec_dec=torch.stack(spk_rec_dec,dim=2) #//////////////////////////ADDED
#         # spk_mem_dec=torch.stack(spk_mem_dec,dim=2) #//////////////////////////ADDED
#         spk_rec2=torch.stack(spk_rec2,dim=4)
#         spk_mem2=torch.stack(spk_mem2,dim=4)

#         # out = spk_mem2[:,:,:,:,-1] #return the membrane potential of the output neuron at t = -1 (last t)
#         # out = spk_rec2[:,:,:,:,-1]  #//////////////////////////ADDED #  Size of out: torch.Size([250, 1, 32, 32])
#         # out = spk_rec_dec[:,:,-1] #//////////////////////////ADDED  # Size of out: torch.Size([250, 1, 32, 32])
#         # out = spk_rec_dec[:,0,:,-1] #//////////////////////////ADDED  # Size of out: torch.Size([250, 5, 32])
#         # out = spk_rec_dec[:,0,0,:,-1] #//////////////////////////ADDED  # Size of out: torch.Size([250, 32])
#         # out = spk_rec_dec[:,-1] #//////////////////////////ADDED  Size of out: torch.Size([250, 5, 32, 32])
#         # out = spk_rec2[0,0,:,:] #//////////////////////////ADDED Now working with [250, 1, 32, 32]) because it is (torch.Size([32, 32, 5]))
#         out = spk_mem2[:,:,:,:,-1]  #//////////////////////////ADDED #  Size of out: torch.Size([250, 1, 32, 32])--------------- for one digit at the last time t

#         # print("Size of out:", out.size())

#         # Save the out_en tensor
#         self.out_en = out_en
#         self.out = out

#         return out, out_en


#     def encode(self,x):
#       spk_latent_x, mem_latent_x = self.encoder(x)
#       return spk_latent_x, mem_latent_x, syn1, memsyn1

#     def decode(self,x):
#         spk_x, mem_x = self.linearNet(x) #convert latent dimension back to total size of features in encoder final layer
#         spk_x2, mem_x2 = self.decoder(spk_x)
#         return spk_x2, mem_x2

### Working with "out" and "out_en":


In [ ]:
# class SAE(nn.Module):
#     def __init__(self):
#         super().__init__()
#         # Encoder
#         self.encoder = nn.Sequential(
#                             nn.Conv2d(1, 32, 3,padding = 1,stride=2), # Conv Layer 1
#                             nn.BatchNorm2d(32),
#                             snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True,threshold=thresh),
#                             nn.Conv2d(32, 64, 3,padding = 1,stride=2), # Conv Layer 2
#                             nn.BatchNorm2d(64),
#                             snn.Synaptic(alpha=alpha, beta=beta_syn, spike_grad=spike_grad, init_hidden=True,threshold=thresh), #SNN TORCH LIF NEURON
#                             nn.Conv2d(64, 128, 3,padding = 1,stride=2), # Conv Layer 3
#                             nn.BatchNorm2d(128),
#                             snn.Synaptic(alpha=alpha, beta=beta_syn, spike_grad=spike_grad, init_hidden=True,threshold=thresh), #SNN TORCH LIF NEURON
#                             nn.Flatten(start_dim = 1, end_dim = 3), #Flatten convolutional output
#                             nn.Linear(128*4*4, latent_dim), # Fully connected linear layer
#                             snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, output=True,threshold=thresh)
#                             )
#         self.latent_dim = latent_dim #dimensions of the encoded z-space data
#         self.linearNet= nn.Sequential(
#                                       nn.Linear(latent_dim,128*4*4),
#                                       snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, output=True,threshold=thresh))

#         # Decoder:
#         self.decoder = nn.Sequential(
#                             nn.Unflatten(1,(128,4,4)), #Unflatten data from 1 dim to tensor of 128 x 4 x 4
#                             snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True,threshold=thresh),
#                             nn.ConvTranspose2d(128, 64, 3,padding = 1,stride=(2,2),output_padding=1),
#                             nn.BatchNorm2d(64),
#                             snn.Synaptic(alpha=alpha, beta=beta_syn, spike_grad=spike_grad, init_hidden=True,threshold=thresh),
#                             nn.ConvTranspose2d(64, 32, 3,padding = 1,stride=(2,2),output_padding=1),
#                             nn.BatchNorm2d(32),
#                             snn.Synaptic(alpha=alpha, beta=beta_syn, spike_grad=spike_grad, init_hidden=True,threshold=thresh),
#                             nn.ConvTranspose2d(32, 1, 3,padding = 1,stride=(2,2),output_padding=1),
#                             snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True,output=True,threshold=20000) #so membrane can be trained
#                             )
#     def forward(self, x):
#         utils.reset(self.encoder) #need to reset the hidden states of LIF
#         utils.reset(self.decoder)
#         utils.reset(self.linearNet)

#     #-----------------------------encode
#         spk_mem=[];
#         spk_rec=[];
#         spk_rec_syn=[];
#         encoder_mem=[];
#         spk_rec_dec=[];
#         spk_mem_dec=[];
#         for step in range(num_steps): #for t in time
#             spk_x, mem_x = self.encode(x) #Output spike trains and neuron membrane states
#             # print("spk_x in spk_x, mem_x = self.encode(x): ",spk_x.size())    #                       torch.Size([250, 32])
#             # print("mem_x in spk_x, mem_x = self.encode(x): ",mem_x.size())    #                       torch.Size([250, 32])
#             spk_rec.append(spk_x)
#             spk_mem.append(mem_x)
#             # print("len of spk_rec in spk_rec.append(spk_x): ",len(spk_rec))
#             # print("len of spk_mem in spk_rec.append(mem_x): ",len(spk_mem))

#         spk_rec=torch.stack(spk_rec,dim=2) # stack spikes in second tensor dimension # ----------------spk_rec in torch.stack(spk_rec,dim=2):  torch.Size([250, 32, 5])
#         # print("----------------spk_rec in torch.stack(spk_rec,dim=2): ",spk_rec.size())
#         spk_mem=torch.stack(spk_mem,dim=2) # stack membranes in second tensor dimension # ----------------spk_mem in torch.stack(spk_mem,dim=2):  torch.Size([250, 32, 5])
#         # print("----------------spk_mem in torch.stack(spk_mem,dim=2): ",spk_mem.size())
#         # out_en = spk_rec[0, :,:] #//////////////////////////////////////ADDED------------ latent dim (32) and time (num_steps)(5) # out_en in spk_rec[0, :,:]:----------- torch.Size([32, 5])
#         # print("out_en in spk_rec[0, :,:]:-----------" , out_en.size())
#         out_en = spk_rec[...,step]
#         # print("spk_rec[...,step]:-----------" , spk_rec[...,step].size()) # spk_rec[...,step]:----------- torch.Size([250, 32])       input of the latent and then decoder




#     #------------------------------decode
#         spk_mem2=[];
#         spk_rec2=[];
#         decoded_x=[];
#         spk_x_dec=[];
#         mem_x_dec=[];
#         for step in range(num_steps): #for t in time                                   from decoder: ([250, 1, 32, 32])
#             x_recon, x_mem_recon = self.decode(spk_rec[...,step])                             # x_recon in self.decode(spk_rec[...,step]): torch.Size([250, 1, 32, 32])
#             # print("x_recon in self.decode(spk_rec[...,step]):" , x_recon.size())            # x_mem_recon in self.decode(spk_rec[...,step]): torch.Size([250, 1, 32, 32])
#             # print("x_mem_recon in self.decode(spk_rec[...,step]):" , x_mem_recon.size())

#             spk_rec2.append(x_recon)
#             spk_mem2.append(x_mem_recon)
#             # print("len of spk_rec2.append(x_recon)",len(spk_rec2))
#             # print("len of spk_mem2.append(x_recon)",len(spk_mem2))

#         spk_rec2=torch.stack(spk_rec2,dim=4)
#         spk_mem2=torch.stack(spk_mem2,dim=4)
#         # print("spk_rec2 in torch.stack(spk_rec2,dim=4):", spk_rec2.size())          # spk_rec2 in torch.stack(spk_rec2,dim=4): torch.Size([250, 1, 32, 32, 5])
#         # print("spk_mem2 in torch.stack(spk_rec2,dim=4):", spk_mem2.size())          # spk_mem2 in torch.stack(spk_rec2,dim=4): torch.Size([250, 1, 32, 32, 5])

#         out = spk_mem2[:,:,:,:,-1]  #//////////////////////////ADDED #  Size of out: torch.Size([250, 1, 32, 32])--------------- for one digit at the last time t

#         # self.out_en = out_en
#         # self.out = out

#         return out, out_en


#     def encode(self,x):
#       spk_latent_x, mem_latent_x = self.encoder(x)
#       return spk_latent_x, mem_latent_x

#     def decode(self,x):
#         spk_x, mem_x = self.linearNet(x) #convert latent dimension back to total size of features in encoder final layer
#         spk_x2, mem_x2 = self.decoder(spk_x)
#         return spk_x2, mem_x2

In [29]:
class SAE(nn.Module):
    def __init__(self):
        super().__init__()


        # encoder_layers = [
        #     ('conv1', nn.Conv2d(1, 32, 3, padding=1, stride=2)),  # Conv Layer 1
        #     ('batchnorm1', nn.BatchNorm2d(32)),
        #     ('leaky1', snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, output=True, threshold=thresh)),
        #     ('conv2', nn.Conv2d(32, 64, 3, padding=1, stride=2)),  # Conv Layer 2
        #     ('batchnorm2', nn.BatchNorm2d(64)),
        #     ('synaptic1', snn.Synaptic(alpha=alpha, beta=beta_syn, spike_grad=spike_grad, init_hidden=True, output=True, threshold=thresh)),  # SNN TORCH LIF NEURON
        #     ('conv3', nn.Conv2d(64, 128, 3, padding=1, stride=2)),  # Conv Layer 3
        #     ('batchnorm3', nn.BatchNorm2d(128)),
        #     ('synaptic2', snn.Synaptic(alpha=alpha, beta=beta_syn, spike_grad=spike_grad, init_hidden=True, output=True, threshold=thresh)),  # SNN TORCH LIF NEURON
        #     ('flatten', nn.Flatten(start_dim=1, end_dim=3)),  # Flatten convolutional output
        #     ('linear', nn.Linear(128 * 4 * 4, latent_dim)),  # Fully connected linear layer
        #     ('leaky2', snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, output=True, threshold=thresh))
        # ]


        # Encoder
        self.encoder = nn.Sequential(
                            nn.Conv2d(1, 32, 3,padding = 1,stride=2), # Conv Layer 1
                            nn.BatchNorm2d(32),
                            snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True,threshold=thresh),
                            nn.Conv2d(32, 64, 3,padding = 1,stride=2), # Conv Layer 2
                            nn.BatchNorm2d(64),
                            snn.Synaptic(alpha=alpha, beta=beta_syn, spike_grad=spike_grad, init_hidden=True, threshold=thresh), #SNN TORCH LIF NEURON
                            nn.Conv2d(64, 128, 3,padding = 1,stride=2), # Conv Layer 3
                            nn.BatchNorm2d(128),
                            snn.Synaptic(alpha=alpha, beta=beta_syn, spike_grad=spike_grad, init_hidden=True,threshold=thresh), #SNN TORCH LIF NEURON
                            nn.Flatten(start_dim = 1, end_dim = 3), #Flatten convolutional output
                            nn.Linear(128*4*4, latent_dim), # Fully connected linear layer
                            snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, output=True,threshold=thresh)
                            )


        self.latent_dim = latent_dim #dimensions of the encoded z-space data
        self.linearNet= nn.Sequential(
                                      nn.Linear(latent_dim,128*4*4),
                                      snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, output=True,threshold=thresh))

        # Decoder:
        self.decoder = nn.Sequential(
                            nn.Unflatten(1,(128,4,4)), #Unflatten data from 1 dim to tensor of 128 x 4 x 4
                            snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True,threshold=thresh),
                            nn.ConvTranspose2d(128, 64, 3,padding = 1,stride=(2,2),output_padding=1),
                            nn.BatchNorm2d(64),
                            snn.Synaptic(alpha=alpha, beta=beta_syn, spike_grad=spike_grad, init_hidden=True,threshold=thresh),
                            nn.ConvTranspose2d(64, 32, 3,padding = 1,stride=(2,2),output_padding=1),
                            nn.BatchNorm2d(32),
                            snn.Synaptic(alpha=alpha, beta=beta_syn, spike_grad=spike_grad, init_hidden=True,threshold=thresh),
                            nn.ConvTranspose2d(32, 1, 3,padding = 1,stride=(2,2),output_padding=1),
                            snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True,output=True,threshold=20000) #so membrane can be trained
                            )
    def forward(self, x):
        utils.reset(self.encoder) #need to reset the hidden states of LIF
        utils.reset(self.decoder)
        utils.reset(self.linearNet)

    #-----------------------------encode
        spk_mem=[];
        spk_rec=[];
        spk_rec_syn=[];
        encoder_mem=[];
        spk_rec_dec=[];
        spk_mem_dec=[];
        enc5_rec = [];


     #------------------------------ intermediate layers

        # for step in range(num_steps):
        #     enc5 = self.encoder[5](x)             #  enc5 shape: torch.Size([250, 1, 32, 32])
        #     enc5_rec.append(enc5)
        # enc5_rec = torch.stack(enc5_rec, dim=2)            #   enc5_rec size: torch.Size([250, 1, 5, 32, 32])
        # Enc_syn_1 = enc5_rec[:, :, -1]                      # #   torch.Size([250, 1, 32, 32])

     #------------------------------ encode
        for step in range(num_steps):
            spk_x, mem_x = self.encoder(x)              # spk_x size: ([250, 32])  ,   mem_x size: ([250, 32])  , x.shape : torch.Size([250, 1, 32, 32])
            spk_rec.append(spk_x)
            spk_mem.append(mem_x)

        spk_rec=torch.stack(spk_rec,dim=2) # stack spikes in second tensor dimension # ----------------spk_rec in torch.stack(spk_rec,dim=2):  torch.Size([250, 32, 5])
        spk_mem=torch.stack(spk_mem,dim=2) # stack membranes in second tensor dimension # ----------------spk_mem in torch.stack(spk_mem,dim=2):  torch.Size([250, 32, 5])
        out_en = spk_rec[...,step]

#ADDED---------------------------------------------
        # Apply Poisson noise to spike trains
        noise_level = 0.1  # Adjust the noise level
        noise = torch.poisson(out_en * noise_level)
        # Add noise to the signal
        noisy_spk_rec = out_en + noise

# -------------------------------------------------
        # print("out_en= spk_rec[...,step]:-----------" , spk_rec[...,step].size()) # spk_rec[...,step]:----------- torch.Size([250, 32])       input of the latent and then decoder

     #------------------------------decode
        spk_mem2=[];
        spk_rec2=[];
        decoded_x=[];
        spk_x_dec=[];
        mem_x_dec=[];
        for step in range(num_steps): #for t in time                           #        from decoder: ([250, 1, 32, 32])
            x_recon, x_mem_recon = self.decode(spk_rec[...,step])
            spk_rec2.append(x_recon)
            spk_mem2.append(x_mem_recon)

        spk_rec2=torch.stack(spk_rec2,dim=4)
        spk_mem2=torch.stack(spk_mem2,dim=4)

        out = spk_mem2[:,:,:,:,-1]

        self.out_en = out_en
        self.out = out

        return out, out_en
        # return out, Enc_syn_1

    def encode(self,x):
      spk_latent_x, mem_latent_x = self.encoder(x)
      return spk_latent_x, mem_latent_x


    def decode(self,x):
        spk_x, mem_x = self.linearNet(x) #convert latent dimension back to total size of features in encoder final layer
        spk_x2, mem_x2 = self.decoder(spk_x)
        return spk_x2, mem_x2



In [30]:
# shape = (250, 1, 32, 32)

# # Create a random tensor with the specified shape
# x = torch.randn(shape)

# print(x.size())
# print(x.shape)

# print(x.size(0))

# print(x.view(x.size(0), -1).size())
# x=x.view(x.size(0), -1)
# print("x new.      ",x.size())

# print("len(x)----------",len(x))


# # Assuming x is your tensor of size [250, 1024]
# new_shape = (-1, 32)  # -1 means "whatever is needed to keep the total number of elements the same"
# reshaped_x = x.view(new_shape)

# print("x new.      ",x.size())




## ALL LEAKY

In [31]:
# class SAE(nn.Module):
#     def __init__(self):
#         super().__init__()

#              # Encoder
#         self.enc0=    nn.Conv2d(1, 32, 3, padding=1, stride=2)
#         self.enc1=    nn.BatchNorm2d(32)
#         self.enc2=    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, output=True, threshold=thresh)
#         self.enc3=    nn.Conv2d(32, 64, 3, padding=1, stride=2)
#         self.enc4=    nn.BatchNorm2d(64)
#         self.enc5=    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, output=True, threshold=thresh)
#         self.enc6=    nn.Conv2d(64, 128, 3, padding=1, stride=2)
#         self.enc7=    nn.BatchNorm2d(128)
#         self.enc8=    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, output=True, threshold=thresh)
#         self.enc9=    nn.Flatten(start_dim=1, end_dim=3)
#         self.enc10=   nn.Linear(128 * 4 * 4, latent_dim)
#         self.enc11=   snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, output=True, threshold=thresh)

#         self.latent_dim = latent_dim

#         self.linearNet0= nn.Linear(latent_dim,128*4*4)
#         self.linearNet1=snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, output=True,threshold=thresh)


#         # Decoder:
#         self.dec0=    nn.Unflatten(1,(128,4,4)) #Unflatten data from 1 dim to tensor of 128 x 4 x 4
#         self.dec1=    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, output=True,threshold=thresh)
#         self.dec2=    nn.ConvTranspose2d(128, 64, 3,padding = 1,stride=(2,2),output_padding=1)
#         self.dec3=    nn.BatchNorm2d(64)
#         self.dec4=    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, output=True, threshold=thresh)
#         self.dec5=    nn.ConvTranspose2d(64, 32, 3,padding = 1,stride=(2,2),output_padding=1)
#         self.dec6=    nn.BatchNorm2d(32)
#         self.dec7=    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, output=True, threshold=thresh)
#         self.dec8=    nn.ConvTranspose2d(32, 1, 3,padding = 1,stride=(2,2),output_padding=1)
#         self.dec9=    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True,output=True,threshold=20000) #so membrane can be trained
#        # self.dec9=    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True,output=True,threshold=thresh) #---------------------------------------------- ADDED


#     def forward(self, x):
#         utils.reset(self.enc0)
#         utils.reset(self.enc1)
#         utils.reset(self.enc2)
#         utils.reset(self.enc3)
#         utils.reset(self.enc4)
#         utils.reset(self.enc5)
#         utils.reset(self.enc6)
#         utils.reset(self.enc7)
#         utils.reset(self.enc8)
#         utils.reset(self.enc9)
#         utils.reset(self.enc10)
#         utils.reset(self.enc11)
#         utils.reset(self.linearNet0)
#         utils.reset(self.linearNet1)
#         utils.reset(self.dec0)
#         utils.reset(self.dec1)
#         utils.reset(self.dec2)
#         utils.reset(self.dec3)
#         utils.reset(self.dec4)
#         utils.reset(self.dec5)
#         utils.reset(self.dec6)
#         utils.reset(self.dec7)
#         utils.reset(self.dec8)
#         utils.reset(self.dec9)

#         # # -------Initialize hidden states at t=0

#         x2_rec = []
#         x5_rec = []
#         x8_rec = []
#         x11_rec = []

#         d1_rec = []
#         d4_rec = []
#         d7_rec = []
#         d9_rec = []
#         d9_rec_mem = []

#         for step in range(num_steps):
#             #------------------------------ encoder:
#             x0=self.enc0(x)
#             x1=self.enc1(x0)
#             x2=self.enc2(x1) #Leaky
#             # print("len x2: ", len(x2))
#             # print("x2.size: ", x2.size())
#             x3=self.enc3(x2)
#             x4=self.enc4(x3)
#             x5=self.enc5(x4) #Leaky
#             x6=self.enc6(x5)
#             x7=self.enc7(x6)
#             x8=self.enc8(x7) #Leaky
#             x9=self.enc9(x8)
#             x10=self.enc10(x9)
#             x11=self.enc11(x10) #Leaky
#             # print("len x11: ", len(x11))
#             # print("x11[0].size: ", x11[0].size())         torch.Size([250, 32])
#             # print("x11[1].size: ", x11[1].size())         torch.Size([250, 32])


#             x2_rec.append(x2)              #    x2 size:  torch.Size([250, 32, 16, 16])
#             x5_rec.append(x5)                 #x5 size:  torch.Size([250, 64, 8, 8])
#             x8_rec.append(x8)              #  x8 size:  torch.Size([250, 128, 4, 4])
#             x11_rec.append(x11[0])
#             # print("x11 length: ",len(x11))                        x11 length:  2
#             # print("x11[1] length: ",len(x11[1]))               #   x11[1] length:  250
#             # print("x11[0] length: ",len(x11[0]))               #   x11[0] length:  250
#             # print("x11_rec length: ",len(x11_rec))                x11_rec length:  1 >> 2 >> 3 >> 4 >> 5

#             #-------------------------------------------------
#         x2_rec = torch.stack(x2_rec, dim=2)
#         x5_rec = torch.stack(x5_rec, dim=2)
#         x8_rec = torch.stack(x8_rec, dim=2)
#         x11_rec = torch.stack(x11_rec, dim=2)                         #       torch.Size([250, 32, 5])

#         x2_rec = x2_rec[:, :, -1]                                               # =========  x2_rec[:, :, -1].size() torch.Size([250, 32, 16, 16])
#         x5_rec = x5_rec[:, :, -1]                                               # =========  x5_rec[:, :, -1].size() torch.Size([250, 64, 8, 8])
#         # print("=========  x5_rec[:, :, -1].size()", x5_rec.size())
#         x8_rec = x8_rec[:, :, -1]                                               #  ========= x8_rec[:, :, -1].size() torch.Size([250, 128, 4, 4])
#         # print("=========  x8_rec[:, :, -1].size()", x8_rec.size())
#         out_en = x11_rec[...,step]
#         # print("========= out_en: x11_rec[...,step] size: ", out_en.size())

#         # x11_rec = x11_rec[:, :, -1]             #  =========  x11_rec[:, :, -1].size() torch.Size([250, 32])

#         for step in range(num_steps):
#             #------------------------------ latent:
#             # LN0=self.linearNet0(x11[...,step])         # means keep all dimensions of x11 up to the last one (which is usually the time dimension), and then selecting the data at the step position along that dimension.
#             LN0=self.linearNet0(x11_rec[...,step])
#             # print(" LN0=self.linearNet0(x11_rec[...,step])======", LN0.size())       # torch.Size([250, 2048])
#             # print("type of LN0",type(LN0))
#             # print("length of LN0",len(LN0))
#             LN1=self.linearNet1(LN0)
#             # print("LN1=self.linearNet1(LN0) size:", len(LN1))
#             # print("LN1[0].size()", LN1[0].size())          #  LN1[0].size() torch.Size([250, 2048])
#             # print("LN1[1].size()", LN1[1].size())          #  LN1[1].size() torch.Size([250, 2048])

#             # print("type of LN1",type(LN1))
#             # print("---------------length of LN1",len(LN1))                      # length of LN1: 2
#             # print("length of LN1",len(LN1[0]))                                  # length of LN1[0]: 250
#             # print("length of LN1",len(LN1[1]))                                  # length of LN1[1]: 250
#             #------------------------------ decoder:
#             # d0=self.dec0(LN1[...,step])
#             d0=self.dec0(LN1[0])
#             # print("type of d0",type(d0))
#             # print("length of d0",len(d0))
#             # print("Size of d0", LN1[0].size())              # Size of d0 torch.Size([250, 128, 4, 4])
#             # print("Size of d0", LN1[1].size())              # Size of d0 torch.Size([250, 128, 4, 4])

#             d1=self.dec1(d0)     #Leaky
#             d2=self.dec2(d1)
#             d3=self.dec3(d2)
#             d4=self.dec4(d3)     #Leaky
#             d5=self.dec5(d4)
#             d6=self.dec6(d5)
#             d7=self.dec7(d6)     #Leaky
#             d8=self.dec8(d7)
#             d9=self.dec9(d8)     #Leaky
#             # d9,d9_mem =self.dec9(d8)     #Leaky
#             # print("            d9[0]=self.dec9(d8)", d9[0].size())  #   torch.Size([250, 1, 32, 32])
#             # print("            d9[1]=self.dec9(d8)", d9[1].size())  #   torch.Size([250, 1, 32, 32])

#             # x_recon=self.dec9(d8)
#             #----------------------------------------

#             #-----------decoder:
#             d1_rec.append(d1)
#             # print("length of d1_rec.append(d1)",len(d1_rec))
#             d4_rec.append(d4)
#             # print("length of d4_rec.append(d4)",len(d4_rec))
#             d7_rec.append(d7)
#             # print("length of d7_rec.append(d7)",len(d7_rec))
#             d9_rec.append(d9[0])
#             # print("-----------d9 length: ", len(d9))                                   # length of d9: 2
#             # print("d9[0] length: ", len(d9[0]))                                   # length of d9[0]: 250
#             # print("d9[1] length: ", len(d9[1]))                                   # length of d9[1]: 250
#             # print("length of d9_rec.append(d9)",len(d9_rec))
#             d9_rec_mem.append(d9[1])


#        # decoder
#         d1_rec = torch.stack(d1_rec, dim=4)                                              # torch.Size([250, 128, 4, 4, 5])
#         # print("-----------------length of torch.stack(d1_rec, dim=4)",len(d1_rec))
#         # print("------size of torch.stack(d1_rec, dim=4)",d1_rec.size())
#         d4_rec = torch.stack(d4_rec, dim=4)                                              #torch.Size([250, 64, 8, 8, 5])
#         # print("-----------------length of torch.stack(d4_rec, dim=4)",len(d4_rec))
#         # print("------size of torch.stack(d4_rec, dim=4)",d4_rec.size())

#         d7_rec = torch.stack(d7_rec, dim=4)                                              #torch.Size([250, 32, 16, 16, 5])
#         # print("-----------------length of torch.stack(d7_rec, dim=4)",len(d7_rec))
#         # print("------size of torch.stack(d7_rec, dim=4)",d7_rec.size())

#         d9_rec = torch.stack(d9_rec, dim=4)
#         # print("-----------------length of torch.stack(d9_rec, dim=4)",len(d9_rec))
#         # print("------size of torch.stack(d9_rec, dim=4)",d9_rec.size())                  #size of torch.stack(d9_rec, dim=4) torch.Size([250, 1, 32, 32, 5])
#         d9_rec_mem = torch.stack(d9_rec_mem, dim=4)

#        # decoder
#         d1_rec = d1_rec[:, :, :, :, -1]
#         # print("=========  d1_rec[:, :, :, :, -1].size()", d1_rec.size())              #   d1_rec[:, :, :, :, -1].size() torch.Size([250, 128, 4, 4])
#         d4_rec = d4_rec[:, :, :, :, -1]
#         # print("=========  d4_rec[:, :, :, :, -1].size()", d4_rec.size())              #   d4_rec[:, :, :, :, -1].size() torch.Size([250, 64, 8, 8])
#         d7_rec = d7_rec[:, :, :, :, -1]
#         # print("=========  d7_rec[:, :, :, :, -1].size()", d7_rec.size())              #   d7_rec[:, :, :, :, -1].size() torch.Size([250, 32, 16, 16])
#         d9_rec = d9_rec[:, :, :, :, -1]
#         # print("=========  d9_rec[:, :, :, :, -1].size()", d9_rec.size())              #   d9_rec[:, :, :, :, -1].size() torch.Size([250, 1, 32, 32])
#         out = d9_rec_mem[:, :, :, :, -1]

#         # self.out_en = out_en
#         # self.out = out

#         return d9_rec, out_en

#     def get_activation(self, name):
#         def hook(module, input, output):
#             setattr(self, name, output)  # Store the output as an attribute of the model
#         return hook


# # activation = {}
# #     def get_activation(self, name):
# #         def hook(model, input, output):
# #             activation[name] = output.detach()
# #         return hook


In [32]:
# class SAE(nn.Module):
#     def __init__(self):
#         super().__init__()

#              # Encoder
#         self.enc0=    nn.Conv2d(1, 32, 3, padding=1, stride=2)
#         self.enc1=    nn.BatchNorm2d(32)
#         self.enc2=    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, threshold=thresh)
#         self.enc3=    nn.Conv2d(32, 64, 3, padding=1, stride=2)
#         self.enc4=    nn.BatchNorm2d(64)
#         self.enc5=    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, output=True , threshold=thresh)
#         self.enc6=    nn.Conv2d(64, 128, 3, padding=1, stride=2)
#         self.enc7=    nn.BatchNorm2d(128)
#         self.enc8=    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, threshold=thresh)
#         self.enc9=    nn.Flatten(start_dim=1, end_dim=3)
#         self.enc10=   nn.Linear(128 * 4 * 4, latent_dim)
#         self.enc11=   snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, output=True, threshold=thresh)

#         self.latent_dim = latent_dim

#         self.linearNet0= nn.Linear(latent_dim,128*4*4)
#         self.linearNet1=snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, output=True,threshold=thresh)


#         # Decoder:
#         self.dec0=    nn.Unflatten(1,(128,4,4)) #Unflatten data from 1 dim to tensor of 128 x 4 x 4
#         self.dec1=    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True,threshold=thresh)
#         self.dec2=    nn.ConvTranspose2d(128, 64, 3,padding = 1,stride=(2,2),output_padding=1)
#         self.dec3=    nn.BatchNorm2d(64)
#         self.dec4=    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, threshold=thresh)
#         self.dec5=    nn.ConvTranspose2d(64, 32, 3,padding = 1,stride=(2,2),output_padding=1)
#         self.dec6=    nn.BatchNorm2d(32)
#         self.dec7=    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, threshold=thresh)
#         self.dec8=    nn.ConvTranspose2d(32, 1, 3,padding = 1,stride=(2,2),output_padding=1)
#         self.dec9=    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True,output=True,threshold=20000) #so membrane can be trained
#        # self.dec9=    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True,output=True,threshold=thresh) #---------------------------------------------- ADDED


#     def forward(self, x):
#         utils.reset(self.enc0)
#         utils.reset(self.enc1)
#         utils.reset(self.enc2)
#         utils.reset(self.enc3)
#         utils.reset(self.enc4)
#         utils.reset(self.enc5)
#         utils.reset(self.enc6)
#         utils.reset(self.enc7)
#         utils.reset(self.enc8)
#         utils.reset(self.enc9)
#         utils.reset(self.enc10)
#         utils.reset(self.enc11)
#         utils.reset(self.linearNet0)
#         utils.reset(self.linearNet1)
#         utils.reset(self.dec0)
#         utils.reset(self.dec1)
#         utils.reset(self.dec2)
#         utils.reset(self.dec3)
#         utils.reset(self.dec4)
#         utils.reset(self.dec5)
#         utils.reset(self.dec6)
#         utils.reset(self.dec7)
#         utils.reset(self.dec8)
#         utils.reset(self.dec9)


#         x2_rec = []
#         x5_rec = []
#         x8_rec = []
#         x11_rec = []

#         d1_rec = []
#         d4_rec = []
#         d7_rec = []
#         d9_rec = []
#         d9_rec_mem = []

#         for step in range(num_steps):
#             #------------------------------ encoder:
#             x0=self.enc0(x)
#             x1=self.enc1(x0)
#             x2=self.enc2(x1) #Leaky
#             x3=self.enc3(x2)
#             x4=self.enc4(x3)
#             x5, x5mem =self.enc5(x4) #Leaky
#             x6=self.enc6(x5)
#             x7=self.enc7(x6)
#             x8=self.enc8(x7) #Leaky
#             x9=self.enc9(x8)
#             x10=self.enc10(x9)
#             x11=self.enc11(x10) #Leaky

#             x2_rec.append(x2)
#             x5_rec.append(x5)
#             x8_rec.append(x8)
#             x11_rec.append(x11[0])

#             #-------------------------------------------------
#         x2_rec = torch.stack(x2_rec, dim=2)
#         x5_rec = torch.stack(x5_rec, dim=2)
#         x8_rec = torch.stack(x8_rec, dim=2)
#         x11_rec = torch.stack(x11_rec, dim=2)                         #
#         x2_rec = x2_rec[:, :, -1]
#         x5_rec = x5_rec[:, :, -1]
#         x8_rec = x8_rec[:, :, -1]
#         out_en = x11_rec[...,step]
#         # x11_rec = x11_rec[:, :, -1]             #  =========  x11_rec[:, :, -1].size() torch.Size([250, 32])

#         for step in range(num_steps):
#             #------------------------------ latent:
#             LN0=self.linearNet0(x11_rec[...,step])
#             LN1=self.linearNet1(LN0)
#             #------------------------------ decoder:
#             # d0=self.dec0(LN1[...,step])
#             d0=self.dec0(LN1[0])
#             d1=self.dec1(d0)     #Leaky
#             d2=self.dec2(d1)
#             d3=self.dec3(d2)
#             d4=self.dec4(d3)     #Leaky
#             d5=self.dec5(d4)
#             d6=self.dec6(d5)
#             d7=self.dec7(d6)     #Leaky
#             d8=self.dec8(d7)
#             d9=self.dec9(d8)     #Leaky
#             #----------------------------------------

#             #-----------decoder:
#             d1_rec.append(d1)
#             d4_rec.append(d4)
#             d7_rec.append(d7)
#             d9_rec.append(d9[0])
#             d9_rec_mem.append(d9[1])


#        # decoder
#         d1_rec = torch.stack(d1_rec, dim=4)
#         d4_rec = torch.stack(d4_rec, dim=4)
#         d7_rec = torch.stack(d7_rec, dim=4)
#         d9_rec = torch.stack(d9_rec, dim=4)
#         d9_rec_mem = torch.stack(d9_rec_mem, dim=4)

#        # decoder
#         d1_rec = d1_rec[:, :, :, :, -1]
#         d4_rec = d4_rec[:, :, :, :, -1]
#         d7_rec = d7_rec[:, :, :, :, -1]
#         d9_rec = d9_rec[:, :, :, :, -1]
#         out = d9_rec_mem[:, :, :, :, -1]

#         # self.out_en = out_en
#         # self.out = out

#         return d9_rec, out_en

#     # def get_activation(self, name):
#     #     def hook(module, input, output):
#     #         setattr(self, name, output)  # Store the output as an attribute of the model
#     #     return hook

## for each layer

In [33]:
# # Training and Testing
# from torchvision.utils import save_image

# spike_recordings = []
# train_ber_rec = []
# test_ber_rec = []
# threshold_Real = 0.5
# threshold_Recon = 0.5

# def train(network, trainloader, opti, epoch):
#     network=network.train()
#     train_loss_hist=torch.zeros((1), dtype=dtype, device=device)
#     train_avg_loss_rec=[]

#     for batch_idx, (real_img, labels) in enumerate(trainloader):
#         opti.zero_grad()
#         real_img = real_img.to(device)
#         labels = labels.to(device)

#         # print("real_img size", real_img.size())      #    real_img size                # -------------------------------------------------ADDED
#         out, out_en = network(real_img)
#         x_recon, out_en = network(real_img)
#         # print("out_en size", out_en.size())      #                  # -------------------------------------------------ADDED
#         # print("x_recon size", x_recon.size())      #                  # -------------------------------------------------ADDED


#         #Calculate loss
#         loss_val = torch.zeros((1), dtype=dtype, device=device)
#         for step in range(num_steps):
#           loss_val += F.mse_loss(x_recon, real_img)                  #.view(1, -1)

#         train_loss_hist += (loss_val.item())/num_steps
#         avg_loss=train_loss_hist.mean()

#         # # # ---------------------
#         print(f'Train[{epoch}/{max_epoch}][{batch_idx}/{len(trainloader)}] Loss: {loss_val.item()}')

#         loss_val.backward()        #

#         opti.step()
#         train_loss_rec.append(loss_val.item())

#         #Save reconstructed images every at the end of the epoch
#         if batch_idx == len(trainloader)-1:
#             utls.save_image((real_img+1)/2, f'figures/training/epoch{epoch}_finalbatch_inputs.png')
#             utls.save_image((x_recon+1)/2, f'figures/training/epoch{epoch}_finalbatch_recon.png')
#             train_auc = auc(np.arange(len(train_loss_rec)), train_loss_rec)

#     # return loss_val, train_loss_rec, train_auc, d9_rec, out_en  #              # -------------------------------------------------ADDED
#     return loss_val, train_loss_rec, train_auc, out, out_en  #              # -------------------------------------------------ADDED


# # For Testing, not doing backpropagate, therefore no gradients are required and we use torch.no_grad():
# #Testing Loop
# def test(network, testloader, opti, epoch):
#     network=network.eval()
#     test_loss_hist=[]
#     test_avg_loss_rec=[]
#     test_avg_loss_hist = []

#     spk_rec_test = []
#     with torch.no_grad(): #no gradient this time
#         for batch_idx, (real_img, labels) in enumerate(testloader):
#             real_img = real_img.to(device)#
#             labels = labels.to(device)
#             # x11_rec, d9_rec = network(real_img)             # -------------------------------------------------ADDED
#             out, out_en = network(real_img)  # Pass data into network and return reconstructed image and spk_rec
#             x_recon, out_en = network(real_img)  # Pass data into network and return reconstructed image and spk_rec
#             # average Loss:
#             loss_val = torch.zeros((1), dtype=dtype, device=device)
#             for step in range(num_steps):
#               loss_val += F.mse_loss(x_recon, real_img)
#             avg_loss=loss_val/num_steps
#             test_loss_hist.append(loss_val.item())

#             real_img_binary = (real_img > threshold_Real).float()
#             x_recon_binary = (x_recon > threshold_Recon).float()
#             bit_errors = torch.sum(torch.abs(real_img_binary - x_recon_binary))
#             total_pixels = real_img_binary.numel()  # Total number of pixels in the images
#             bit_error_rate = bit_errors.item() / total_pixels
#             test_ber_rec.append(bit_error_rate)  # Append BER to the list

#             # Save binary images
#             Error_bin = (torch.abs(x_recon_binary - real_img_binary))

#             if batch_idx == len(testloader)-1:
#               if epoch in [0, 25, 49]:
#                 save_image(real_img_binary, f'figures/binarytesting/ep{epoch}_inputs_binary.png')
#                 save_image(x_recon_binary, f'figures/binarytesting/ep{epoch}_recon_binary.png')
#                 save_image(Error_bin, f'figures/binarytesting/ep{epoch}_Error_bin.png')
#             # -------------------------------------------------------------------------------------------------

#             print(f'Test[{epoch}/{max_epoch}][{batch_idx}/{len(testloader)}]  Loss: {loss_val.item()}, '  f'BER (test): {bit_error_rate}')

#             test_loss_rec.append(loss_val.item())

#             if batch_idx == len(testloader)-1:
#                 utls.save_image((real_img+1)/2, f'figures/testing/epoch{epoch}_finalbatch_inputs.png')
#                 utls.save_image((x_recon+1)/2, f'figures/testing/epoch{epoch}_finalbatch_recons.png')
#                 test_auc = auc(np.arange(len(test_loss_rec)), test_loss_rec)

#     # return loss_val, test_loss_rec, test_auc, out, out_en                     # -------------------------------------------------ADDED
#     # return loss_val, test_loss_rec, test_auc, x11_rec, d9_rec  #              # -------------------------------------------------ADDED
#     return loss_val, test_loss_rec, test_auc, out, out_en



In [34]:
# # Training and Testing
# # using MSE loss to compare the reconstructed image (x_recon) with the original image (real_img)
# from torchvision.utils import save_image

# spike_recordings = []
# train_ber_rec = []
# test_ber_rec = []
# threshold_Real = 0.5
# threshold_Recon = 0.5

# def train(network, trainloader, opti, epoch):
#     network=network.train()
#     train_loss_hist=torch.zeros((1), dtype=dtype, device=device)
#     train_avg_loss_rec=[]

#     for batch_idx, (real_img, labels) in enumerate(trainloader):
#         opti.zero_grad()
#         real_img = real_img.to(device)
#         labels = labels.to(device)
#         # d9_rec, out_en = network(real_img)             # -------------------------------------------------ADDED
#         # print("real_img size", real_img.size())      #    real_img size torch.Size([250, 1, 32, 32])                # -------------------------------------------------ADDED

#         # out, out_en = network(real_img)  # Pass data into network and return reconstructed image and spk_rec
#         x_recon, out_en = network(real_img)  # Pass data into network and return reconstructed image and spk_rec
#         # print("x_recon size", x_recon.size())      #       x_recon size torch.Size([250, 1, 32, 32])
#         # print("out size", out.size())             #       out size torch.Size([250, 32])

#         #Calculate loss
#         loss_val = torch.zeros((1), dtype=dtype, device=device)
#         for step in range(num_steps):
#           loss_val += F.mse_loss(x_recon, real_img)                  #.view(1, -1)

#             # Clone the loss_val tensor to avoid in-place modification

#         train_loss_hist += (loss_val.item())/num_steps
#         avg_loss=train_loss_hist.mean()

#         # # ---------------------------- Calculate Bit Error Rate (BER)
#         real_img_binary = (real_img > threshold_Real).float()
#         x_recon_binary = (x_recon > threshold_Recon).float()
#         bit_errors = torch.sum(torch.abs(real_img_binary - x_recon_binary))
#         total_pixels = real_img_binary.numel()  # Total number of pixels in the images
#         bit_error_rate = bit_errors.item() / total_pixels
#         train_ber_rec.append(bit_error_rate)  # Append BER to the list

#         # Save binary images
#         Error_bin = (torch.abs(x_recon_binary - real_img_binary))
#         if batch_idx == len(trainloader)-1:
#           if epoch in [0, 25, 49]:
#             utls.save_image(real_img_binary, f'figures/binarytraining/ep{epoch}_inputs_binary.png')
#             utls.save_image(x_recon_binary, f'figures/binarytraining/ep{epoch}_recon_binary.png')
#             utls.save_image(Error_bin, f'figures/binarytraining/ep{epoch}_Error_bin.png')
#         print(f'Train[{epoch}/{max_epoch}][{batch_idx}/{len(trainloader)}] Loss: {loss_val.item()}, ' f'BER : {bit_error_rate}')

#         # loss_val += torch.mean(loss_val)  # Accumulate the loss             # -------------------------------------------------ADDED
#         loss_val.backward()         #
#         # loss_val.backward(retain_graph=True)        #

#         opti.step()
#         train_loss_rec.append(loss_val.item())

#         #Save reconstructed images every at the end of the epoch
#         if batch_idx == len(trainloader)-1:
#             utls.save_image((real_img+1)/2, f'figures/training/epoch{epoch}_finalbatch_inputs.png')
#             utls.save_image((x_recon+1)/2, f'figures/training/epoch{epoch}_finalbatch_recon.png')
#             train_auc = auc(np.arange(len(train_loss_rec)), train_loss_rec)

#     # loss_val.backward()                                            # -------------------------------------------------ADDED

#     # return loss_val, train_loss_rec, train_auc , out, out_en  #, spk_rec_batches#, train_avg_loss_rec, #avg_loss #, train_loss_hist
#     # return loss_val, train_loss_rec, train_auc   #
#     return loss_val, train_loss_rec, train_auc, d9_rec, out_en  #              # -------------------------------------------------ADDED


# # For Testing, not doing backpropagate, therefore no gradients are required and we use torch.no_grad():
# #Testing Loop
# def test(network, testloader, opti, epoch):
#     network=network.eval()
#     test_loss_hist=[]
#     test_avg_loss_rec=[]
#     test_avg_loss_hist = []

#     spk_rec_test = []
#     with torch.no_grad(): #no gradient this time
#         for batch_idx, (real_img, labels) in enumerate(testloader):
#             real_img = real_img.to(device)#
#             labels = labels.to(device)
#             # x11_rec, d9_rec = network(real_img)             # -------------------------------------------------ADDED
#             # out, out_en = network(real_img)  # Pass data into network and return reconstructed image and spk_rec
#             x_recon, out_en = network(real_img)  # Pass data into network and return reconstructed image and spk_rec
#             # average Loss:
#             loss_val = torch.zeros((1), dtype=dtype, device=device)
#             for step in range(num_steps):
#               loss_val += F.mse_loss(x_recon, real_img)
#             avg_loss=loss_val/num_steps
#             test_loss_hist.append(loss_val.item())

#             real_img_binary = (real_img > threshold_Real).float()
#             x_recon_binary = (x_recon > threshold_Recon).float()
#             bit_errors = torch.sum(torch.abs(real_img_binary - x_recon_binary))
#             total_pixels = real_img_binary.numel()  # Total number of pixels in the images
#             bit_error_rate = bit_errors.item() / total_pixels
#             test_ber_rec.append(bit_error_rate)  # Append BER to the list

#             # Save binary images
#             Error_bin = (torch.abs(x_recon_binary - real_img_binary))

#             if batch_idx == len(testloader)-1:
#               if epoch in [0, 25, 49]:
#                 save_image(real_img_binary, f'figures/binarytesting/ep{epoch}_inputs_binary.png')
#                 save_image(x_recon_binary, f'figures/binarytesting/ep{epoch}_recon_binary.png')
#                 save_image(Error_bin, f'figures/binarytesting/ep{epoch}_Error_bin.png')
#             # -------------------------------------------------------------------------------------------------

#             print(f'Test[{epoch}/{max_epoch}][{batch_idx}/{len(testloader)}]  Loss: {loss_val.item()}, '  f'BER (test): {bit_error_rate}')

#             test_loss_rec.append(loss_val.item())

#             if batch_idx == len(testloader)-1:
#                 utls.save_image((real_img+1)/2, f'figures/testing/epoch{epoch}_finalbatch_inputs.png')
#                 utls.save_image((x_recon+1)/2, f'figures/testing/epoch{epoch}_finalbatch_recons.png')
#                 test_auc = auc(np.arange(len(test_loss_rec)), test_loss_rec)

#     # return loss_val, test_loss_rec, test_auc, out, out_en                     # -------------------------------------------------ADDED
#     # return loss_val, test_loss_rec, test_auc, x11_rec, d9_rec  #              # -------------------------------------------------ADDED
#     return loss_val, test_loss_rec, test_auc  #



In [35]:
# Training and Testing
# using MSE loss to compare the reconstructed image (x_recon) with the original image (real_img)
from torchvision.utils import save_image

spike_recordings = []
train_ber_rec = []
test_ber_rec = []
threshold_Real = 0.5
threshold_Recon = 0.5
std_dev=0.1


def train(network, trainloader, opti, epoch):
    network=network.train()
    train_loss_hist=torch.zeros((1), dtype=dtype, device=device)
    train_avg_loss_rec=[]

    for batch_idx, (real_img, labels) in enumerate(trainloader):
        opti.zero_grad()
        real_img = real_img.to(device)
        labels = labels.to(device)

        # -------------Add Gaussian noise to the input images
        real_img = real_img + torch.randn_like(real_img) * std_dev

        out, out_en = network(real_img)  # Pass data into network and return reconstructed image and spk_rec
        x_recon, out = network(real_img)  # Pass data into network and return reconstructed image and spk_rec.  #        x_recon size torch.Size([250, 1, 32, 32]) ,  #        out size torch.Size([250, 32])
        #Calculate loss
        loss_val = torch.zeros((1), dtype=dtype, device=device)
        for step in range(num_steps):
          loss_val += F.mse_loss(x_recon, real_img)                  #.view(1, -1)
        train_loss_hist += (loss_val.item())/num_steps
        avg_loss=train_loss_hist.mean()

        # # ---------------------------- Calculate Bit Error Rate (BER)
        real_img_binary = (real_img > threshold_Real).float()
        x_recon_binary = (x_recon > threshold_Recon).float()
        bit_errors = torch.sum(torch.abs(real_img_binary - x_recon_binary))
        total_pixels = real_img_binary.numel()  # Total number of pixels in the images
        bit_error_rate = bit_errors.item() / total_pixels
        train_ber_rec.append(bit_error_rate)  # Append BER to the list

        # Save binary images
        Error_bin = (torch.abs(x_recon_binary - real_img_binary))

        if batch_idx == len(trainloader)-1:
          if epoch in [0, 25, 49]:
            utls.save_image(real_img_binary, f'figures/binarytraining/ep{epoch}_inputs_binary.png')
            utls.save_image(x_recon_binary, f'figures/binarytraining/ep{epoch}_recon_binary.png')
            utls.save_image(Error_bin, f'figures/binarytraining/ep{epoch}_Error_bin.png')
        print(f'Train[{epoch}/{max_epoch}][{batch_idx}/{len(trainloader)}] Loss: {loss_val.item()}, ' f'BER : {bit_error_rate}')

        loss_val.backward()
        opti.step()
        train_loss_rec.append(loss_val.item())

        #Save reconstructed images every at the end of the epoch
        if batch_idx == len(trainloader)-1:
            utls.save_image((real_img+1)/2, f'figures/training/epoch{epoch}_finalbatch_inputs.png')
            utls.save_image((x_recon+1)/2, f'figures/training/epoch{epoch}_finalbatch_recon.png')
            train_auc = auc(np.arange(len(train_loss_rec)), train_loss_rec)

    return loss_val, train_loss_rec, train_auc , out, out_en  #, spk_rec_batches#, train_avg_loss_rec, #avg_loss #, train_loss_hist
    # return loss_val, train_loss_rec, train_auc   #


# For Testing, not doing backpropagate, therefore no gradients are required and we use torch.no_grad():
#Testing Loop
def test(network, testloader, opti, epoch):
    network=network.eval()
    test_loss_hist=[]
    test_avg_loss_rec=[]
    test_avg_loss_hist = []

    spk_rec_test = []
    with torch.no_grad(): #no gradient this time
        for batch_idx, (real_img, labels) in enumerate(testloader):
            real_img = real_img.to(device)#
            labels = labels.to(device)
            out, out_en = network(real_img)  # Pass data into network and return reconstructed image and spk_rec
            x_recon , out = network(real_img)  # Pass data into network and return reconstructed image and spk_rec
            # average Loss:
            loss_val = torch.zeros((1), dtype=dtype, device=device)
            for step in range(num_steps):
              loss_val += F.mse_loss(x_recon, real_img)
            avg_loss=loss_val/num_steps
            test_loss_hist.append(loss_val.item())

            real_img_binary = (real_img > threshold_Real).float()
            x_recon_binary = (x_recon > threshold_Recon).float()
            bit_errors = torch.sum(torch.abs(real_img_binary - x_recon_binary))
            total_pixels = real_img_binary.numel()  # Total number of pixels in the images
            bit_error_rate = bit_errors.item() / total_pixels
            test_ber_rec.append(bit_error_rate)  # Append BER to the list

            # Save binary images
            Error_bin = (torch.abs(x_recon_binary - real_img_binary))

            if batch_idx == len(testloader)-1:
              if epoch in [0, 25, 49]:
                save_image(real_img_binary, f'figures/binarytesting/ep{epoch}_inputs_binary.png')
                save_image(x_recon_binary, f'figures/binarytesting/ep{epoch}_recon_binary.png')
                save_image(Error_bin, f'figures/binarytesting/ep{epoch}_Error_bin.png')
            # -------------------------------------------------------------------------------------------------

            print(f'Test[{epoch}/{max_epoch}][{batch_idx}/{len(testloader)}]  Loss: {loss_val.item()}, '  f'BER (test): {bit_error_rate}')

            test_loss_rec.append(loss_val.item())

            if batch_idx == len(testloader)-1:
                utls.save_image((real_img+1)/2, f'figures/testing/epoch{epoch}_finalbatch_inputs.png')
                utls.save_image((real_img+(torch.randn_like(real_img) * std_dev)+1)/2, f'figures/testing/(real_img+noise)_epoch{epoch}_finalbatch_inputs.png')
                utls.save_image((x_recon+1)/2, f'figures/testing/epoch{epoch}_finalbatch_recons.png')
                test_auc = auc(np.arange(len(test_loss_rec)), test_loss_rec)

    return loss_val, test_loss_rec, test_auc, out, out_en                     # -------------------------------------------------ADDED
    # return loss_val, test_loss_rec, test_auc  #

for batch_spikes in spike_recordings:
    print(batch_spikes)


In [36]:
input_size = 32 #resize of mnist data (optional)

#setup GPU
dtype = torch.float
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# neuron and simulation parameters
spike_grad = surrogate.atan(alpha=2.0)  # alternate surrogate gradient fast_sigmoid(slope=25)

train_loss_rec = []
test_loss_rec = []
train_loss_record = []
test_loss_record = []
train_avg_loss_rec=[]
test_avg_loss_rec=[]

  # Synaptic current and membrane potential decay exponentially with rates of alpha and beta
alpha=0.9
beta_syn=0.0001
# beta_syn=0.9

beta =0.9

num_steps=5
latent_dim = 32 #dimension of latent layer (how compressed we want the information)
thresh=1    #spiking threshold (lower = more spikes are let through)
epochs=50
# epochs=5
max_epoch=epochs

  #Define Network and optimizer
net=SAE()
net = net.to(device)
optimizer = torch.optim.AdamW(net.parameters(),
                            lr=0.0001,
                            betas=(0.9, 0.999),
                            weight_decay=0.001)



activation = {}
# def get_activation(name):
#     def hook(model, input, output):
#         activation[name] = output.detach()
#     return hook


def get_activation(name):
    def hook(model, input, output):
        if isinstance(output, tuple):
            activation[name] = [out.detach() for out in output]
        else:
            activation[name] = output.detach()
    return hook


# net.encoder[5].register_forward_hook(get_activation('encoder[5]'))


## for saving the out_en after each epoch:

In [ ]:
checkpoint_path = "Saved_Trained_Checkpoints/"
Output_Spikes = "Output_Spikes/"
Enc_syn_Spikes = "Enc_syn_Spikes/"
Intermediate_Lyrs = "Intermediate_Lyrs/"
epoch_activations_list = [] # Create a list to store activations for each epoch
epoch_activations = {}


# ///////////////////////////////////

# Define hook_layers and hook_names
hook_layers = [net.encoder[2], net.encoder[5], net.encoder[8], net.encoder[11], net.decoder[1], net.decoder[4], net.decoder[7], net.decoder[9]]
hook_names = ["Enc_Lk1", "Enc_syn1", "Enc_syn2", "Enc_Lk2", "Dec_Lk1", "Dec_syn1", "Dec_syn2", "Dec_Lk2"]

# Create an empty dictionary to store activations
epoch_activations = {}

# Register hooks for capturing activations
hooks = []
for i, layer in enumerate(hook_layers):
    hook_fn = get_activation(hook_names[i])
    hooks.append(layer.register_forward_hook(hook_fn))

# Run training and testing
for e in range(epochs):
    train_loss = train(net, train_loader, optimizer, e)
    train_avg_loss_rec.append(sum(train_loss_rec) / len(train_loader))

    test_loss = test(net, test_loader, optimizer, e)
    test_avg_loss_rec.append(sum(test_loss_rec) / (len(test_loader)))

    # #---------------------------------------------------------- Access the out_en tensor
    out_en = net.out_en
    out_en_numpy = out_en.cpu().detach().numpy()
    # Save with a different name for each epoch
    out_en_filename = Output_Spikes + f"out_en_epoch_{e + 1}.npy"
    np.save(out_en_filename, out_en_numpy)

    # #-----------------------------------------------------------Access the out tensor
    out = net.out
    out_numpy = out.cpu().detach().numpy()
    # Save with a different name for each epoch
    out_filename = Output_Spikes + f"out_epoch_{e + 1}.npy"
    np.save(out_filename, out_numpy)
    # -------------------------------------------------------------------- Intermediate Layers
# Check if the current epoch is a multiple of 10
    if (e + 1) % 10 == 0:
        # Save the epoch_activations dictionary to a file
        activations_path = Intermediate_Lyrs +  f"activations_epoch_{e + 1}.pkl"
        with open(activations_path, 'wb') as file:
            pickle.dump(epoch_activations, file)
    # Capture activations for the current epoch
    epoch_activations[e] = {}
    for i, name in enumerate(hook_names):
        epoch_activations[e][name] = activation.get(name, None)  # Use get to avoid KeyError




# beta_syn=0.0001


Train[0/50][0/240] Loss: 8.006251335144043, BER : 0.86759765625
Train[0/50][1/240] Loss: 8.01241683959961, BER : 0.86846875
Train[0/50][2/240] Loss: 8.04263687133789, BER : 0.87267578125
Train[0/50][3/240] Loss: 8.040471076965332, BER : 0.87218359375
Train[0/50][4/240] Loss: 7.965984344482422, BER : 0.86468359375
Train[0/50][5/240] Loss: 8.055792808532715, BER : 0.8756953125
Train[0/50][6/240] Loss: 8.02262020111084, BER : 0.87193359375
Train[0/50][7/240] Loss: 8.061013221740723, BER : 0.87718359375
Train[0/50][8/240] Loss: 7.977132797241211, BER : 0.8683671875
Train[0/50][9/240] Loss: 7.974903583526611, BER : 0.868078125
Train[0/50][10/240] Loss: 7.9753265380859375, BER : 0.8694609375
Train[0/50][11/240] Loss: 8.03271484375, BER : 0.8691484375
Train[0/50][12/240] Loss: 8.015864372253418, BER : 0.86117578125
Train[0/50][13/240] Loss: 8.188153266906738, BER : 0.85907421875
Train[0/50][14/240] Loss: 8.378402709960938, BER : 0.84477734375
Train[0/50][15/240] Loss: 8.393698692321777, BER :